**3) 잠재 디리클레 할당(LDA) 실습2**

1) 뉴스 기사 제목 데이터에 대한 이해

In [ ]:
import pandas as pd
import urllib.request

urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", filename="abcnews-date-text.csv")
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)

print(len(data))
print()
print(data[:5])
print()

In [ ]:
text = data[['headline_text']]
text = text.copy() 
print(text.head(5))
print()

2) 텍스트 전처리

In [ ]:
# 토큰화

import nltk
# nltk.download('punkt')

text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

print(text.head(5))
print()

In [ ]:
# 불용어 제거
# nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK 로 부터 불용어를 받아옵니다.
text['headline_text'] = text['headline_text'].apply(lambda x: [item for item in x if item not in stop_words]) # 불용어 제거

print(text.head(5))
print()

In [ ]:
# 표제어 추출
# nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

print(text.head(5))
print()

In [ ]:
# 길이가 3 이하인 단어 제거
tokenized_doc = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 3])

print(tokenized_doc[:5])
print()

3) TF-IDF 행렬 만들기

In [ ]:
# 역토큰화 (토큰화 작업을 되돌림)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

text['headline_text'] = detokenized_doc # 다시 text['headline_text']에 재저장

print(text['headline_text'][:5])
print()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000) # 상위 1000개의 단어를 보존
X= vectorizer.fit_transform(text['headline_text'])
print(X.shape) # TF-IDF 행렬의 크기 확인

4) 토픽 모델링

In [49]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)

In [50]:
lda_top=lda_model.fit_transform(X)

In [ ]:
print(lda_model.components_)
print(lda_model.components_.shape) 

In [ ]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)